# Image Matching Challenge 2022

## Epipolar Geometry and Fundamental Matrix

In this notebook we will look at the coherence between the Epipolar Geometry of a stereo camera and the given Fundamental Matrix. This should illustrate the understanding of the task. We are going to visualize the Epipolar Lines to simplify the correspondance problem of two pictures with different point of views.

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# root directory for image matching challenge

ROOT = '../input/image-matching-challenge-2022/'

# generate a list with all scenes showed in the dataset

scenes = os.listdir(ROOT+'train')
scenes.remove('LICENSE.txt')
scenes.remove('scaling_factors.csv')

# scaling factory for the scenes

sf = pd.read_csv(ROOT+'train/scaling_factors.csv')

df = pd.DataFrame()
for scene in scenes:
    
    # load calibration data and assign the corresponding scaling factors and scenes
    
    scene_df = pd.read_csv(ROOT+'train/'+scene+'/calibration.csv')
    scene_df['scaling_factor'] = sf[sf.scene==scene].values[0][1]
    scene_df['scene'] = scene
    
    # generate the complete image path
    
    scene_df['image_path'] = ROOT+'train/'+scene+'/images/'+scene_df['image_id']+'.jpg'
    
    df = df.append(scene_df, ignore_index=True)
    
# load all pair_covisibility.csv files
# with this DataFrame we can look at the different
# pairs and can access the fundamental matrix

pairs = pd.DataFrame()
for scene in scenes:
    scene_df = pd.read_csv(ROOT+'train/'+scene+'/pair_covisibility.csv')
    scene_df['scene'] = scene
    pairs = pairs.append(scene_df, ignore_index=True)

## Plot an example image

Let's look at an example image. The picture shows the Brandenburg Gate in Berlin, Germany. The images were taken from two different angles and different distances, but they show the same object.

In [ ]:
# IMAGE PLOTTING FUNCTIONS

# extract image file from dataset DataFrame
def read_img(image_name):
    pth = df.image_path[df.image_id==image_name].values[0]
    img = cv2.imread(pth)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

# plot two images side by side
def plot_image_pair(img1, img2):
    fig, axs = plt.subplots(1, 2, figsize=(20,10))
    axs[0].imshow(img1)
    axs[1].imshow(img2)
    axs[0].axis('off'); axs[1].axis('off')
    
# PLOT THE EXAMPLE PICTURE
smpl = pairs.loc[1383416]
imgs = smpl.pair.split('-')
img1 = read_img(imgs[0])
img2 = read_img(imgs[1])
plot_image_pair(img1, img2)

## Fundamental Matrix

The fundamental matrix $F$ of the image pair is given and describes the geometric relation of the two images. The two different viewing angles create a stereo camera perspective, which makes a three-dimensional view possible.

Let's have a first look at the matrix. The matrix is a 3x3 dimensional array and describes the relationship between perspective 1 and 2 in the form shown.

In [ ]:
np.set_printoptions(precision=3, suppress=True)
def string_to_matrix(s):
    s = s.split(' ')
    s = list(map(float, s))
    return np.array(s).reshape(3,3)

F = string_to_matrix(smpl.fundamental_matrix)
print(F)

## Calibration Matrix

The calibration matrices (```camera_intrinsics```) of the cameras are also given and include the intrinsic parameters. They are an important part of the geometric relationship.

In [ ]:
K1 = df.camera_intrinsics[df.image_id==imgs[0]].values[0]
K1 = string_to_matrix(K1)
print(K1)

K2 = df.camera_intrinsics[df.image_id==imgs[1]].values[0]
K2 = string_to_matrix(K2)
print(K2)

## Essential Matrix

With the given fundamental matrix $F$ and the calibration matrices $K$ of the individual cameras, the essential matrix $E$ can be calculated. This is also suitable for describing the perspective relationships. 

<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#E4E4E4;
           font-size:100%;
           font-family:Verdana;
           letter-spacing:0.5px">
        <p style="padding: 10px;
              text-align:center;
              color:black;">
            $E=(K_1)^T \cdot F \cdot K_2$
        </p>
    </div>

In [ ]:
E = K1.transpose() * F * K2
print(E)

## Epipolar Lines

The relationship between two images that show the same object from different angles can be described using epipolar geometry. 

A main topic of image recognition is the solution of the correspondence problem. This deals with the matching of a point seen on both images of a stereo camera. A pixel on the left image must be matched with the corresponding pixel on the right image. 

With the fundamental matrix it is possible to simplify the correspondence problem. We can use it to calculate the epipolar line on the right image from one pixel of the left image. The point we are looking for lies on this line. With this Calculation, the problem could be changed from a 2D image to a 1D line and simplifies the matching considerably.

Calculate the line with a matrix multiplication of the fundamental matrix and the point on the other image:
<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#E4E4E4;
           font-size:100%;
           font-family:Verdana;
           letter-spacing:0.5px">
        <p style="padding: 10px;
              text-align:center;
              color:black;">
            $L = F \cdot p$
        </p>
    </div>

In [ ]:
# point on first (left) image with the pixel coordinates
point = np.array([523, 235, 1])

# calculate the line on the second (right) image
line = F @ point
line = line.reshape(-1, 3)
print(line[0])

In [ ]:
# 1st image
img1 = cv2.circle(img1, point[:2], 10, (255,0,0), -1)

# 2nd image
def draw_lines(img, line):
    for i,l in enumerate(line):
        p1 = np.array([0, (-l[2]/l[1])], dtype=np.int32)
        p2 = np.array([img.shape[1], (-l[0]/l[1])*img.shape[1]+(-l[2]/l[1])], dtype=np.int32)
        img = cv2.line(img, tuple(p1), tuple(p2), (255,0,0), 3)
    return img

img2 = draw_lines(img2, line)
plot_image_pair(img1, img2)

As you can see in the image, the line in the right image passes through the corresponding point. To solve the correspondence problem, only the line must now be searched for the target point instead of each individual pixel in the whole image.